# Classification

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle
import time

# Model selection
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.utils import class_weight

# Classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.svm import SVC

# performance metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay
from sklearn.metrics import average_precision_score

In [ ]:
df = pd.read_csv('../data/interim/covid_articles_tfidf.csv')
df.head()

In [ ]:
df.topic_area.value_counts()

Since the number of articles per topic area are not balanced, I will filter out the tags with low frequency. Alternatively, I may merge the excluded tags with the closest tag with higher frequency.

In [ ]:
min_samples = 10000

filter_topics = df['topic_area'].value_counts().to_frame()
filter_topics = filter_topics[filter_topics['topic_area']>min_samples].index
df_majority = df[df['topic_area'].isin(filter_topics)].drop(columns='title')
print(df_majority.shape)
df_majority.head()

### Encode labels
We can use onehotencoder or label encoder to transform the labels. However, scikit learn can digest text labels and I will use text labels for now. The script below will be used if I need encoded labels in the future.

In [ ]:
def label_enc(array):
    enc = LabelEncoder()
    #y = df_majority.topic_area.values
    array = enc.fit_transform(array)
    n_classes = len(enc.classes_[0])
    return array, n_classes

In [ ]:
def ordinal_enc(array):
    enc = OrdinalEncoder()
    #y = df_majority.topic_area.values
    array = enc.fit_transform(array.reshape(-1,1))
    n_classes = len(enc.categories_[0])
    return array, n_classes

In [ ]:
def onehot_enc(array):
    enc = OneHotEncoder()
    #y = df_majority.topic_area.values.reshape(-1,1)
    array = enc.fit_transform(array.reshape(-1,1)).toarray()
    n_classes = len(enc.categories_[0])
    #print((enc.categories_))
    return array, n_classes, enc.categories_[0]

### ROC Curve for Multiclass Classification

ROC curves typically feature true positive rate on the Y axis, and false positive rate on the X axis. This means that the top left corner of the plot is the “ideal” point - a false positive rate of zero, and a true positive rate of one. This is not very realistic, but it does mean that a larger area under the curve (AUC) is usually better.

I use the following function to generate ROC curve for the exisitng multi-class problem.

In [ ]:
def roc_calc(y_test, y_pred_prob):
    
    y_test_enc, n_classes, _ = onehot_enc(y_test)
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_enc[:, i], y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    return fpr, tpr, roc_auc

In [ ]:
def roc_plot(fpr, tpr, roc_auc):
    plt.figure()
    lw = 2
    plt.plot(fpr[0], tpr[0], color='r',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
    plt.plot([0, 1], [0, 1], color='b', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

### Precision Recall Curve

Intuitively, precision is the ability of the classifier not to label as positive a sample that is negative, and recall is the ability of the classifier to find all the positive samples.

The precision_recall_curve computes a precision-recall curve from the ground truth label and a score given by the classifier by varying a decision threshold. I use the follwoing function to calculate and plot the precision and recall for the existing multi-class problem.

In [ ]:
# For each class
def pre_rec_calc(y_test, y_pred_prob):
    
    y_test_enc, n_classes, _ = onehot_enc(y_test)
    
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test_enc[:, i], y_pred_prob[:, i])
        average_precision[i] = average_precision_score(y_test_enc[:, i], y_pred_prob[:, i])
    
    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(
        y_test_enc.ravel(), y_pred_prob.ravel()
    )
    average_precision["micro"] = average_precision_score(y_test_enc, y_pred_prob, average="micro")
    
    display = PrecisionRecallDisplay(
        recall=recall["micro"],
        precision=precision["micro"],
        average_precision=average_precision["micro"],
    )
    display.plot()
    _ = display.ax_.set_title("Micro-averaged over all classes")
    
    return precision, recall, average_precision

In [ ]:
# setup plot details
def pre_rec_plot(precision, recall, average_precision):
    
    y_test_enc, n_classes, categories = onehot_enc(y_test)
    
    colors = cycle(["navy", "turquoise", "darkorange", "cornflowerblue", "teal"])
    
    _, ax = plt.subplots(figsize=(7, 8))
    
    f_scores = np.linspace(0.2, 0.8, num=4)
    lines, labels = [], []
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        (l,) = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
        plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))
        
    display = PrecisionRecallDisplay(
        recall=recall["micro"],
        precision=precision["micro"],
        average_precision=average_precision["micro"],
    )
    
    display.plot(ax=ax, name="Micro-average precision-recall", color="gold")
        
    for i, color in zip(range(n_classes), colors):
        display = PrecisionRecallDisplay(
            recall=recall[i],
            precision=precision[i],
            average_precision=average_precision[i],
        )
        display.plot(ax=ax, name=f"Precision-recall for class {categories[i]}", color=color)
    # add the legend for the iso-f1 curves
    handles, labels = display.ax_.get_legend_handles_labels()
    handles.extend([l])
    labels.extend(["iso-f1 curves"])
    # set the legend and the axes
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.legend(handles=handles, labels=labels, loc="best")
    ax.set_title("Extension of Precision-Recall curve to multi-class")
    
    plt.show()

### Train and Test Data

In [ ]:
X = df_majority.drop(columns='topic_area')
y = df_majority.topic_area.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)

In [ ]:
weights_classes = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

In [ ]:
weight_dict = dict(zip(np.unique(y_train), weights_classes))
weight_dict

In [ ]:
weights = [weight_dict[x] for x in y_train]

## Classifiers
I will apply NaiveBayes, Logistic Regression, and SGD classifiers on the data.

### Naive Bayes

In [ ]:
# Naive Bayes
clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)#, sample_weight=weights)
y_pred = clf_nb.predict(X_test)
y_pred_prob = clf_nb.predict_proba(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
fpr, tpr, roc_auc = roc_calc(y_test, y_pred_prob)
roc_plot(fpr, tpr, roc_auc)

In [ ]:
precision, recall, average_precision = pre_rec_calc(y_test, y_pred_prob)
pre_rec_plot(precision, recall, average_precision)

### Logistic Regression

In [ ]:
clf_lr = LogisticRegressionCV()
clf_lr.fit(X_train, y_train)
y_pred = clf_lr.predict(X_test)
y_pred_prob = clf_lr.predict_proba(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
fpr, tpr, roc_auc = roc_calc(y_test, y_pred_prob)
roc_plot(fpr, tpr, roc_auc)

In [ ]:
precision, recall, average_precision = pre_rec_calc(y_test, y_pred_prob)
pre_rec_plot(precision, recall, average_precision)

### SGD Classifier

In [ ]:
clf_sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)
clf_sgd.fit(X_train, y_train)
y_pred = clf_sgd.predict(X_test)

print(classification_report(y_test, y_pred))

## Pipeline
The next step I create a pipeline to put all these steps together and find an optimum hyperparamter space.

In [ ]:
df_normal_text = pd.read_csv('../data/interim/covid_articles_normalized.csv')

min_samples = 10000

filter_topics = df_normal_text['topic_area'].value_counts().to_frame()
filter_topics = filter_topics[filter_topics['topic_area']>min_samples].index
df_select_topics = df_normal_text[df_normal_text['topic_area'].isin(filter_topics)].drop(columns='title')

In [ ]:
X = df_select_topics.content.tolist()
y = df_select_topics.topic_area.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(min_df=3)),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultinomialNB())
])

In [ ]:
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'vect__max_features': [10000, 15000, 30000, None],
    'vect__max_df': (0.5, 0.75, 1.0),
    'tfidf__use_idf': (True, False),
    'clf__alpha': [1, 1e-1, 1e-2],
    'clf__sample_weight':[None, weights],
}

In [ ]:
search = GridSearchCV(pipeline, param_grid, n_jobs=-1, verbose=1)

In [ ]:
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(param_grid)
t0 = time()

search.fit(data.data, data.target)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % search.best_score_)
print("Best parameters set:")
best_parameters = search.best_estimator_.get_params()

for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))